# DC Line dispatch with pandapower OPF
This is an introduction into the usage of the pandapower optimal power flow with dc lines.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf_dcline.png" width="100%">

We first create this network in pandapower:

In [1]:
import pandapower as pp
from numpy import array
net = pp.create_empty_network()

b1 = pp.create_bus(net, 380)
b2 = pp.create_bus(net, 380)
b3 = pp.create_bus(net, 380)
b4 = pp.create_bus(net, 380)
b5 = pp.create_bus(net, 380)

l1 = pp.create_line(net, b1, b2, 30, "490-AL1/64-ST1A 380.0")
l2 = pp.create_line(net, b3, b4, 20, "490-AL1/64-ST1A 380.0")
l3 = pp.create_line(net, b4, b5, 20, "490-AL1/64-ST1A 380.0")

dcl1 = pp.create_dcline(net, name="dc line", from_bus=b2, to_bus=b3, p_kw=0.2e6, loss_percent=1.0, 
                  loss_kw=500, vm_from_pu=1.01, vm_to_pu=1.012, max_p_kw=1e6,
                  in_service=True)

eg1 = pp.create_ext_grid(net, b1, 1.02, max_p_kw=0.)
eg2 = pp.create_ext_grid(net, b5, 1.02, max_p_kw=0.)

l1 = pp.create_load(net, bus=b4, p_kw=800e3, controllable = False)

We now run a regular load flow to check out the DC line model:

In [2]:
pp.runpp(net)

The transmission power of the DC line is defined in the loadflow as given by the p_kw parameter, which was set to 0.2 GW:

In [3]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,200000.0,152443.185449,-197500.0,74491.758943,2500.0,1.01,-0.48595,1.012,-0.725627


The losses amount to 2500 kW, which are made up of 500 kW conversion loss and 200 MW * 0.01 = 2 MW transmission losses. The voltage setpoints defined at from and to bus are complied with. 

Now lets define costs for the external grids to run an OPF:

In [4]:
costeg0 = pp.create_polynomial_cost(net, 0, 'ext_grid', array([-.1, 0]))
costeg1 = pp.create_polynomial_cost(net, 1, 'ext_grid', array([-.08, 0]))
net.bus['max_vm_pu'] = 1.5
net.line['max_loading_percent'] = 1000


In [5]:
pp.runopp(net)

hp.pandapower.run - WARNING: The OPF cost definition has changed! Please check out the tutorial 'opf_changes-may18.ipynb' or the documentation!
hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


Since we defined lower costs for Ext Grid 2, it fully services the load:

In [6]:
net.res_ext_grid

,p_kw,q_kvar
0,-500.069543,7787.526681
1,-805091.485151,-628.364177


While the DC line does not transmit any power:

In [7]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,500.066597,7787.487035,-0.065937,-627.07003,500.000659,1.019994,-0.001448,1.013925,-1.563437


If we set the costs of the left grid to a lower value than the right grid and run the loadflow again:

In [8]:
net.polynomial_cost.c.at[costeg0]= array([[-0.08, 0]])
net.polynomial_cost.c.at[costeg1]= array([[-0.1, 0]])
pp.runopp(net)

hp.pandapower.run - WARNING: The OPF cost definition has changed! Please check out the tutorial 'opf_changes-may18.ipynb' or the documentation!
hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


We can see that the power now comes from the left ext_grid:

In [9]:
net.res_ext_grid

,p_kw,q_kvar
0,-821525.335364,7787.52293
1,-0.076326,21048.53705


And is transmitted over the DC line:

In [10]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,813573.861197,-26446.009887,-805023.624948,-21736.255783,8550.236249,1.011014,-2.39987,1.027504,1.522333


We can however see that the lines on the left hand side are now overloaded:

In [11]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,821525.335364,-7787.522930,-813573.861197,26446.009887,7951.474167,18658.486958,1.223760,1.223277,1.223760,127.474948
1,805023.624948,21736.255783,-800001.898584,-10668.046603,5021.726364,11068.209180,1.190800,1.191114,1.191114,124.074380
2,1.898584,10668.046603,0.076326,-21048.537050,1.974910,-10380.490446,0.015882,0.031353,0.031353,3.265926


If we set the maximum line loading to 100% and run the OPF again:

In [12]:
net.line["max_loading_percent"] = 100
pp.runopp(net)

hp.pandapower.run - WARNING: The OPF cost definition has changed! Please check out the tutorial 'opf_changes-may18.ipynb' or the documentation!
hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


We can see that the lines are no longer overloaded:

In [13]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,644488.865237,-682.170542,-639594.579162,6209.563500,4894.286075,5527.392959,0.960000,0.959877,0.960000,100.000008
1,632766.910296,10099.087946,-629647.081603,-7139.222429,3119.828693,2959.865517,0.938624,0.938836,0.938836,97.795377
2,-170352.918397,7139.222429,170582.478966,-16528.481902,229.560569,-9389.259473,0.254211,0.255281,0.255281,26.591815


Because the load is serviced from both grids:

In [14]:
net.res_ext_grid

,p_kw,q_kvar
0,-644488.864003,682.170421
1,-170582.478965,16528.481902


And the DC line transmits only part of the power needed to service the load:

In [15]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,639594.5794,-6209.557071,-632766.910297,-10099.087944,6827.669103,1.012431,-1.875023,1.024385,0.875621


Finally, we can also define transmission costs for the DC line:

In [16]:
costeg1 = pp.create_polynomial_cost(net, 0, 'dcline', array([.03, 0]))
pp.runopp(net)

hp.pandapower.run - WARNING: The OPF cost definition has changed! Please check out the tutorial 'opf_changes-may18.ipynb' or the documentation!
hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


Because the sum of the costs for generating power on the left hand side (0.08) and transmitting it to the right side (0.03) is now larger than for generating on the right side (0.1), the OPF draws as much power from the right side as is possible without violating line loading constraints:

In [17]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,161370.360877,-7787.502930,-161063.562350,-6442.955697,306.798527,-14230.458627,0.240649,0.240554,0.240649,25.067629
1,158973.820074,-4908.697476,-158773.830261,-4527.035241,199.989813,-9435.732717,0.237798,0.237780,0.237798,24.770655
2,-641226.161479,4527.034231,644488.768140,-868.452306,3262.606661,3658.581925,0.959936,0.960000,0.960000,100.000027


In [18]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,161063.56235,6442.955697,-158973.824109,4908.731204,2089.738241,1.018095,-0.467991,1.016201,-0.938714


If we broaden the line loading constraint and run the OPF again:

In [19]:
net.line["max_loading_percent"] = 1000
pp.runopp(net)

hp.pandapower.run - WARNING: The OPF cost definition has changed! Please check out the tutorial 'opf_changes-may18.ipynb' or the documentation!
hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


The load is once again fully serviced by the grid on the right hand side:

In [20]:
net.res_ext_grid

,p_kw,q_kvar
0,-500.199303,7787.496911
1,-805091.355031,-628.373796


And the DC line is in open loop operation:

In [21]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,500.196355,7787.516751,-0.194411,-627.053329,500.001944,1.019994,-0.001448,1.013925,-1.563436


Little consistency check:

In [24]:
net.res_ext_grid.p_kw.at[0]*-0.08 + net.res_ext_grid.p_kw.at[1]*-0.1 + net.res_dcline.p_from_kw.at[0]*0.03

80564.157337965371

In [25]:
net.res_cost

80564.157337965371